In [173]:
import pandas as pd

In [174]:
t_df = pd.read_csv('2020-9/klm1144.csv')
c_df = pd.read_csv('2020-11/klm1144.csv')

In [175]:
def cleaning(t_df, c_df):
    c_df = c_df.loc[:, ~c_df.columns.str.contains('^Unnamed')]
    t_df = t_df.loc[:, ~t_df.columns.str.contains('^Unnamed')]
    if not c_df.dtypes.equals(t_df.dtypes):
        raise ValueError('두 데이터의 타입이 다르다')
    elif not all(c_df.columns.__eq__(t_df.columns)):
        raise ValueError('두 데이터의 컬럼이 다르다')
    else:
        return c_df, t_df
t_df, c_df = cleaning(t_df, c_df)

In [176]:
curr_order = t_df.groupby('campaign_id').sum('payment')
curr_order

,payment
campaign_id,
3005196,654000
3005197,2733000
3046829,1881000
3251512,1352000


In [177]:
compare_order = c_df.groupby('campaign_id').sum('payment')
compare_order

,payment
campaign_id,
3005196,1010000
3005197,1858000
3046829,992000


In [178]:
curr_order = t_df.groupby('campaign_id').sum('payment')
compare_order = c_df.groupby('campaign_id').sum('payment')

try:
    curr_order = curr_order.drop('기타')
except KeyError:
    pass
try:
    compare_order = compare_order.drop('기타')
except KeyError:
    pass

curr_order['profit'] = curr_order.payment.subtract(compare_order.payment + 88000, fill_value=0)

In [179]:
a = curr_order.profit / curr_order.payment
a

campaign_id
3005196   -0.678899
3005197    0.287962
3046829    0.425837
3251512    1.000000
dtype: float64

In [180]:
a.map(lambda x: x % -1 if x < 0 else x % 1)

campaign_id
3005196   -0.678899
3005197    0.287962
3046829    0.425837
3251512    0.000000
dtype: float64

In [181]:
    curr_order['profit'] = curr_order.payment.subtract(compare_order.payment + 88000, fill_value=0)
    curr_order['profit'] = curr_order.profit / curr_order.payment
    curr_order['profit'] = curr_order['profit'].map(lambda x: x % -1 if x < 0 else x % 1)
    curr_order['profit'] = curr_order['profit'] * 100
    curr_order

,payment,profit
campaign_id,,
3005196,654000,-67.889908
3005197,2733000,28.796195
3046829,1881000,42.583732
3251512,1352000,0.000000


In [182]:
    curr_order['profit'] = curr_order.payment.subtract(compare_order.payment + 88000, fill_value=0)

In [183]:
def apply_adv_info(df_by_campaign_id, adv_info):
    adv_info = pd.Series(adv_info, name="address")

    for k in adv_info.index:
        if k not in df_by_campaign_id.index:
            df_by_campaign_id.loc[k] = [np.nan, np.nan]
    df_by_campaign_id = pd.concat([df_by_campaign_id, adv_info], axis=1)
    return df_by_campaign_id

In [184]:
import json
import numpy as np
with open(f"2020-11/adv_info.json", "r") as f:
    adv_info = pd.Series(json.load(f))
adv_info.index = adv_info.index.map(int)

In [185]:
curr_order = apply_adv_info(curr_order, adv_info)

In [186]:
curr_order

,payment,profit,address
3005196,654000,-444000.0,경기도 화성시 매송면 화성로2397번길 49
3005197,2733000,787000.0,경기도 수원시 권선구 삼천병마로 1545
3046829,1881000,801000.0,경기도 수원시 권선구 산업로 174-30
3251512,1352000,1352000.0,경기도 수원시 권선구 매송고색로794번길 249
